# Proyecto Demo Day [Español]

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Import data

In [ ]:
!ls drive/MyDrive/Saturdays\ AI/Dataset

ls: cannot access 'drive/MyDrive/Saturdays AI/Dataset': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Headline es el título de cada noticia
* Category significa que la noticia es verdadera o falsa, es decir:

| 0 | Falso |
|:-:|-------|
| 1 | Verdadero|

1. Headline se cambia a title
2. Categorie cambia a True

In [ ]:
dfmx = pd.read_csv('/content/drive/MyDrive/Saturdays AI/Dataset/df-complete.csv')
dfbo = pd.read_csv('/content/drive/MyDrive/Saturdays AI/Dataset/bolivia.csv')
df = pd.DataFrame(zip(list(dfmx['Headline'])+list(dfbo['Headline']),list(dfmx['Category'])+list(dfbo['Category'])), columns = ["title", "True"])
df

,title,True
0,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",0
1,"La palabra ""haiga"", aceptada por la RAE",0
2,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,0
3,pretenden aprobar libros escolares con conteni...,0
4,UNAM REALIZARÁ PRUEBAS ANTIDOPING A ESTUDIANTE...,0
...,...,...
1555,Santa Cruz: COED restringe circulación hasta l...,1
1556,Loza sobre propuesta de impuesto a cocaleros: ...,1
1557,Intentaron enviar droga impregnada en ropa dep...,1
1558,Granizada en Quillacollo deja daños en 4 insta...,1


## Prepare data

### Preprocessing

In [ ]:
def rschar_title(a_string):
  alphanumeric = ""
  for character in a_string:
    if character.isalnum() or character == ' ':
        alphanumeric += character
  return alphanumeric.title()

sentences = df["title"]
sentences = [(rschar_title(x)) for x in sentences]
# print(sentences)
categorie =np.array(df['True'])
categorie

array([0, 0, 0, ..., 1, 1, 1])

### Separate features

In [ ]:
lst_headers = [x.split(" ") for x in sentences]
headers = []
for i in lst_headers:
  for j in i:
    headers.append(j)

headers = list(set(headers))
data = np.ones((len(sentences),len(headers)))

for [numi, i] in enumerate(sentences):
  for [numj, j] in enumerate(headers):
    # print(i)
    # print(j)
    # print((i.split(" ")).count(j))
    data.itemset((numi, numj),(i.split(" ")).count(j)) #data.itemset(pos,valor)

data =pd.DataFrame(data,columns=headers)
data

,,Ham,5,Fiscalía,Tregua,Colonizar,Vuelven,12,Asevera,Inicia,Tengo,Mdp,Asteroide,Proximas,Britany,Cirujano,Contagiaron,Bs,Olímpicos,Fotos,Chaqueta,Terminando,Recupera,Organización,Verificado,Mí,Beatriz,Chamarra,Saben,Arresto,Bolivia,Aparicio,Juicios,Freud,Multa,Rocha,Albarracín,Ganarían,Movistar,185,...,Asilados,Arquitectos,Sexy,Sentencia,Cuauhtemoc,Unión,Popularidad,Tarifas,Patria,Pobretones,32000,Portuguesa,Celebra,Zamora,Nuevo,Intenta,Pumas,Guaidó,Versión,Portales,Niños,Coches,Policial,Delfina,Cuñado,Interpol,Pensé,Pronóstico,Tarjeta,Plantea,Desde,Italia,Enreda,Crecida,Vallejo,Tilda,Misael,Tendremos,Mar,Prolongan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1556,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Clean data

In [ ]:
freq = pd.DataFrame({'sum': data.sum()})
freq = freq.sort_values(by='sum', ascending=False)
freq = freq.head(25)
freq = list(freq.index)
freq.remove('Number') # Se repite 125 veces
freq.remove('Amlo') # Se repite 82 veces
freq.remove('México') # Se repite 74 veces
freq.remove('Gobierno') # Se repite 57 veces
freq
data = data.drop(freq, axis=1)
headers = data.keys()

### Encode features

In [ ]:
features = data.to_records(index=False)
# print (tuple(features))
X = [list(i) for i in features]

### Separate target

In [ ]:
y = categorie

### Holdout

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Model

### Import libraries

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

### Instance model

In [ ]:
#@markdown * SVM
k = "linear" #@param ["linear", "poly", "rbf", "sigmoid"]
supportvector = SVC(kernel=k) 

#@markdown * Decision Tree
criterio = 'entropy' #@param ['entropy', 'gini']
tree = DecisionTreeClassifier(criterion=criterio, splitter='best',max_depth=7, min_samples_split=4, min_samples_leaf=20, max_features=2)

#@markdown * Logistic regression
logistic = LogisticRegression()

### Fit model

In [ ]:
supportvector.fit(X_train, y_train)
tree.fit(X_train, y_train)
logistic.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Metrics

### Import libraries

In [ ]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

### Confusion matrix

In [ ]:
matrixes = {}
matrixes['Decision tree'] = metrics.confusion_matrix(y_test,tree.predict(X_test))
matrixes['Support Vector Machine'] = metrics.confusion_matrix(y_test,supportvector.predict(X_test))
matrixes['Logistic Regresion'] = metrics.confusion_matrix(y_test,logistic.predict(X_test))

for i in zip(matrixes,matrixes.items()):
  print(i[0]+':')
  print(list(i[1])[1])
  print()

Decision tree:
[[  0 151]
 [  0 317]]

Support Vector Machine:
[[ 50 101]
 [ 33 284]]

Logistic Regresion:
[[ 38 113]
 [ 20 297]]



### Accuracy score

In [ ]:
acc = {}
acc['Decision tree'] = metrics.accuracy_score(y_test,tree.predict(X_test))
acc['Support Vector Machine'] = metrics.accuracy_score(y_test,supportvector.predict(X_test))
acc['Logistic Regresion'] = metrics.accuracy_score(y_test,logistic.predict(X_test))

acc

{'Decision tree': 0.6773504273504274,
 'Logistic Regresion': 0.7158119658119658,
 'Support Vector Machine': 0.7136752136752137}

### F1 score

In [ ]:
fsc = {}
fsc['Decision tree'] = metrics.f1_score(y_test,tree.predict(X_test))
fsc['Support Vector Machine'] = metrics.f1_score(y_test,supportvector.predict(X_test))
fsc['Logistic Regresion'] = metrics.f1_score(y_test,logistic.predict(X_test))

fsc

{'Decision tree': 0.8076433121019109,
 'Logistic Regresion': 0.8170563961485557,
 'Support Vector Machine': 0.8091168091168092}

# Proyecto Demo Day [Inglés]

## Import libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dft = pd.read_csv("/content/drive/MyDrive/Saturdays AI/Dataset/English/True.csv")
dft['True'] = 1
dff = pd.read_csv("/content/drive/MyDrive/Saturdays AI/Dataset/English/Fake.csv")
dff['True'] = 0
df = pd.concat([dft, dff])
df

,title,text,subject,date,True
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to chardge 'much mo...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


### Separate features

In [ ]:
X = df['title']

### Separate target

In [ ]:
y = df['True']

### Holdout

In [ ]:
x_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state = 1)

## Model

### Import libraries

In [ ]:
from sklearn.pipeline import Pipeline

# Multinomial NB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# SVM
from sklearn.svm import LinearSVC

#Passive Aggressive Classifier
from sklearn.linear_model import PassiveAggressiveClassifier

### Pipelines

In [ ]:
#@markdown * Multinomial Naive Bayes
pipe1 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB())
])


#@markdown * SVM
pipe2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC())
])

#@markdown * Passive Agressive Classifier
pipe3 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',  PassiveAggressiveClassifier())
])


### Fit model

In [ ]:
mnb = pipe1.fit(x_train, y_train)
sv = pipe2.fit(x_train, y_train)
pac = pipe3.fit(x_train, y_train)

## Metrics

### Import libraries

In [ ]:
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

### Confusion matrix

In [ ]:
matrixes = {}
matrixes['Multiple Naive Bayes'] = metrics.confusion_matrix(y_test,mnb.predict(X_test))
matrixes['Support Vector Machine'] = metrics.confusion_matrix(y_test,sv.predict(X_test))
matrixes['Passive Aggressive Classifier'] = metrics.confusion_matrix(y_test,pac.predict(X_test))

for i in zip(matrixes,matrixes.items()):
  print(i[0]+':')
  print(list(i[1])[1])
  print()

Multiple Naive Bayes:
[[4525  153]
 [ 353 3949]]

Support Vector Machine:
[[4528  150]
 [ 153 4149]]

Passive Aggressive Classifier:
[[4493  185]
 [ 189 4113]]



### Accuracy score

In [ ]:
acc = {}
acc['Multiple Naive Bayes'] = metrics.accuracy_score(y_test,mnb.predict(X_test))
acc['Support Vector Machine'] = metrics.accuracy_score(y_test,sv.predict(X_test))
acc['Passive Aggressive Classifier'] = metrics.accuracy_score(y_test,pac.predict(X_test))

acc

{'Multiple Naive Bayes': 0.9436525612472161,
 'Passive Aggressive Classifier': 0.9583518930957684,
 'Support Vector Machine': 0.9662583518930957}

### F1 score

In [ ]:
fsc = {}
fsc['Multiple Naive Bayes'] = metrics.f1_score(y_test,mnb.predict(X_test))
fsc['Support Vector Machine'] = metrics.f1_score(y_test,sv.predict(X_test))
fsc['Passive Aggressive Classifier'] = metrics.f1_score(y_test,pac.predict(X_test))

fsc

{'Multiple Naive Bayes': 0.9397905759162303,
 'Passive Aggressive Classifier': 0.9565116279069767,
 'Support Vector Machine': 0.9647715381932334}

# Predict input

In [ ]:
#@markdown # Algoritmos según el idioma
#@markdown |Idioma|Algoritmo 1|Algoritmo 2|Algoritmo 3|
#@markdown |------|-----------|-----------|-----------|
#@markdown |Español|Decision tree|SVM|Logistic regression|
#@markdown |Inglés|Multiple Naive Bayes|SVM|Passive Aggressive Classifier|

language = "English" #@param ["English", "Español"]
algorithm = "SVM" #@param ['Decision tree', 'SVM', 'Logistic regression', 'Multiple Naive Bayes', 'Passive Agressive Classifier']

text = "Mcdonalds arrives in Bolivia in 2020" #@param {type:"string"}

text = (rschar_title(text))
lst_text = []

for i in headers:
  lst_text.append((text.split(" ")).count(i))

if language == 'Español':
  if algorithm == "Decision tree":
    print({0:"Falso", 1:"Verdadero"}[int(tree.predict([np.array(lst_text)]))])
  elif algorithm == "SVM":
    print({0:"Falso", 1:"Verdadero"}[int(supportvector.predict([np.array(lst_text)]))])
  elif algorithm == "Logistic regression":
    print({0:"Falso", 1:"Verdadero"}[int(logistic.predict([np.array(lst_text)]))])
  else:
    print('No se encontró el algoritmo solicitado')
else:
  if algorithm == "Multiple Naive Bayes":
    print({0:"Fake News", 1:"True News"}[int(mnb.predict([text]))])
  elif algorithm == "SVM":
    print({0:"Fake News", 1:"True News"}[int(sv.predict([text]))])
  elif algorithm == "Passive Aggressive Classifier":
    print({0:"Fake News", 1:"True News"}[int(pac.predict([text]))])
  else:
    print('No se encontró el algoritmo solicitado')



Fake News


# WEB

## Install

In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-cp36-none-any.whl size=45216 sha256=ea7d450983b19e6663ea05d91db059b9be6279df2e8908f6a236194a06731d9c
  Stored in directory: /root/.cache/pip/wheels/a2/6e/4e/8b0ae12fb9b8a05715256952cf7609a8ab86285fab99b88c68
Successfully built ws4py


## Connect to Anvil web

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect('7BGB3LV7FB4OF3L2EFRSBLVO-6WMLPS6QLI7QGBJE')

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


## Code send to Anvil Web

In [ ]:
@anvil.server.callable
def test_news(text, lang):
  if lang == 'Inglés':
    return {0:"FALSE", 1:"TRUE"}[int(sv.predict([text]))]
  else:
    alphanumeric = ""
    for character in text:
      if character.isalnum() or character == ' ':
          alphanumeric += character
    text = alphanumeric.title()
    lst_text = []

    for i in headers:
      lst_text.append((text.split(" ")).count(i))
    return {0:"FALSE", 1:"TRUE"}[int(logistic.predict([np.array(lst_text)]))]
    

In [ ]:
anvil.server.wait_forever()
print(text)
print(lang)